### 1. Импорт библиотек и подготовка датафрейма:

In [1]:
import pandas as pd
import random
from datetime import datetime, timedelta
from faker import Faker
import re
import numpy as np

In [2]:
# Создание списка случайного количества покупателей
fake = Faker()
num_buyers = random.randint(1000, 10000)
buyers = [{'buyer_name': fake.name().replace(' ', '_'),
           'num_pay_card': fake.credit_card_number()}
          for _ in range(num_buyers)]

# Добавление записей с "unknown" и None
num_unknown = random.randint(1000, 5000)
for _ in range(num_unknown):
    buyers.append({'buyer_name': 'unknown',
                   'num_pay_card': None})


# Ввод переменных
start_year = int(input("Введите начальный год: "))
end_year = int(input("Введите конечный год: "))
min_amount = float(input("Введите минимальную сумму покупки: "))
max_amount = float(input("Введите максимальную сумму покупки: "))


# Функция для генерации случайного времени от 10 утра до 22 вечера
def random_time(current_date):
    start_time = datetime(current_date.year,
                          current_date.month,
                          current_date.day, 10)
    end_time = datetime(current_date.year,
                        current_date.month,
                        current_date.day, 22)
    return start_time + timedelta(seconds=random
                                  .randint(0, (end_time - start_time).seconds))


# Функция для определения типа карты по её номеру
def determine_card_type(card_number):
    if card_number and re.match(r'^4\d{15}$', card_number):
        return 'Visa'
    elif card_number and re.match(r'^5\d{15}$', card_number):
        return 'MasterCard'
    elif card_number and re.match(r'^6\d{15}$', card_number):
        return 'UnionPay'
    elif card_number and re.match(r'^3[47]\d{13}$', card_number):
        return 'AmericanExpress'
    else:
        return 'Other'


# Генерация покупок для каждого дня
purchases = []
current_date = datetime(start_year, 1, 1, 10, 0, 0)
end_date = datetime(end_year, 12, 31, 22, 0, 0)
while current_date <= end_date:
    num_purchases = random.randint(10, 8642)
    for _ in range(num_purchases):
        buyer = random.choice(buyers)
        type_pay_card = None
        if buyer['num_pay_card']:
            type_pay_card = determine_card_type(buyer['num_pay_card'])
        purchase = {
            'buyer_name': buyer['buyer_name']
            if buyer['num_pay_card'] else 'unknown',
            'type_pay_card': type_pay_card,
            'num_pay_card': buyer['num_pay_card'],
            'purchase_amount': round(random.uniform(min_amount,
                                                    max_amount), 2),
            'purchase_date': random_time(current_date)
        }
        purchases.append(purchase)
    current_date += timedelta(days=1)

# Создание DataFrame
df = pd.DataFrame(purchases)

# Сортировка по дате и времени
df = df.sort_values(by='purchase_date')

df

Введите начальный год:  2020
Введите конечный год:  2023
Введите минимальную сумму покупки:  5
Введите максимальную сумму покупки:  5000


,buyer_name,type_pay_card,num_pay_card,purchase_amount,purchase_date
2540,Amber_Ross,Other,4966164513515,2606.33,2020-01-01 10:00:05
3462,unknown,None,None,680.28,2020-01-01 10:00:06
2827,unknown,None,None,4484.66,2020-01-01 10:00:25
1994,unknown,None,None,3523.36,2020-01-01 10:00:29
1898,unknown,None,None,1280.75,2020-01-01 10:00:34
...,...,...,...,...,...
6385459,unknown,None,None,549.17,2023-12-31 21:57:08
6385445,Jason_Sanders,Visa,4842938078119976,3904.46,2023-12-31 21:57:14
6384469,Steven_Armstrong,Other,38931593140173,2822.17,2023-12-31 21:58:16
6385544,Victoria_Rogers,Other,3501646212799207,987.09,2023-12-31 21:58:26


### 2. Задания к датафрейму:

1) Подсчитать общее количество покупателей
2) Подсчитать общее количество покупателей по типу карты
3) Подсчитать общее количество покупок по типу карты
4) Подсчитать сумму чеков для каждого типа карты
5) Подсчитать средний чек для каждого типа карты
6) Подсчитать количество покупок для каждого года
7) Подсчитать количество покупок для каждого года по типу карты
8) Подсчитать количество покупок, совершенных каждым покупателем
9) Найти среднее количество покупок в день
10) Найти среднее количество покупок в день по типу карты

#### 1. Подсчитать общее количество покупателей

In [3]:
total_buyers = df['buyer_name'].nunique()
print("Общее количество покупателей:", total_buyers)

Общее количество покупателей: 6772


#### 2. Подсчитать общее количество покупателей по типу карты

In [4]:
buyer_count_by_card_type = (df.groupby('type_pay_card')['buyer_name']
                            .nunique()
                            .reset_index()
                            .sort_values(by='buyer_name', ascending=False))
print("Количество покупателей по типу карты оплаты:")
buyer_count_by_card_type

Количество покупателей по типу карты оплаты:


,type_pay_card,buyer_name
2,Other,4440
4,Visa,1143
0,AmericanExpress,614
3,UnionPay,586
1,MasterCard,158


#### 3. Подсчитать общее количество покупок по типу карты

In [5]:
purchase_count_by_card_type = (df.groupby('type_pay_card')
                               .size()
                               .reset_index(name='count')
                               .sort_values(by='count', ascending=False))

print("Количество покупок для каждого типа карты оплаты:")
purchase_count_by_card_type

Количество покупок для каждого типа карты оплаты:


,type_pay_card,count
2,Other,2554751
4,Visa,646299
0,AmericanExpress,343270
3,UnionPay,325828
1,MasterCard,88009


#### 4. Подсчитать сумму чеков для каждого типа карты

In [6]:
total_purchase_amount_by_card_type = (df.groupby('type_pay_card')
                                      ['purchase_amount']
                                      .sum()
                                      .reset_index(name='sum')
                                      .sort_values(by='sum', ascending=False))

print("Сумма чеков для каждого типа карты:")
total_purchase_amount_by_card_type

Сумма чеков для каждого типа карты:


,type_pay_card,sum
2,Other,6.389471e+09
4,Visa,1.618980e+09
0,AmericanExpress,8.592876e+08
3,UnionPay,8.143725e+08
1,MasterCard,2.199783e+08


#### 5. Подсчитать средний чек для каждого типа карты

In [7]:
mean_purchase_amount_by_card_type = np.round(df.groupby('type_pay_card')
                                             ['purchase_amount'].mean()
                                             .reset_index(name='mean')
                                             .sort_values(by='mean',
                                                          ascending=False), 2)

print("Средний чек для каждого типа карты:")
mean_purchase_amount_by_card_type

Средний чек для каждого типа карты:


,type_pay_card,mean
4,Visa,2505.00
0,AmericanExpress,2503.24
2,Other,2501.02
1,MasterCard,2499.50
3,UnionPay,2499.39


#### 6. Подсчитать количество покупок для каждого года

In [8]:
purchase_count_by_year = (df['purchase_date'].dt.year
                          .value_counts()
                          .reset_index(name='sum')
                          .sort_values(by='sum', ascending=False))

print("Количество покупок для каждого года:")
purchase_count_by_year

Количество покупок для каждого года:


,purchase_date,sum
0,2020,1650238
1,2022,1591728
2,2021,1578322
3,2023,1565593


#### 7. Подсчитать количество покупок для каждого года по типу карты

In [9]:
# Копирование датафрейма
year_df = df.copy()

# Извлечь год из столбца с датой покупки
year_df['purchase_date'] = year_df['purchase_date'].dt.year

# Подсчитать количество покупок для каждого года и типа карты оплаты
purchase_count_by_year_and_card_type = (year_df
                                        .groupby(['purchase_date',
                                                  'type_pay_card'])
                                        .size()
                                        .reset_index(name='count')
                                        .sort_values(by=['purchase_date',
                                                         'count'],
                                                     ascending=[True, False]))

print("Количество покупок для каждого года по типу карты:")
purchase_count_by_year_and_card_type

Количество покупок для каждого года по типу карты:


,purchase_date,type_pay_card,count
2,2020,Other,660195
4,2020,Visa,167427
0,2020,AmericanExpress,88116
3,2020,UnionPay,83637
1,2020,MasterCard,22812
7,2021,Other,632753
9,2021,Visa,159309
5,2021,AmericanExpress,85084
8,2021,UnionPay,80589
6,2021,MasterCard,21724


#### 8. Подсчитать количество покупок, совершенных каждым покупателем

In [10]:
# Создать DataFrame с количеством покупок для каждого покупателя
purchase_count_per_buyer = df['buyer_name'].value_counts().reset_index()
purchase_count_per_buyer.columns = ['buyer_name', 'purchase_count']

# Добавить столбец с типом карты для каждого покупателя
purchase_count_per_buyer['type_pay_card'] = (df.groupby('buyer_name')
                                             ['type_pay_card']
                                             .first().values)

print("Количество покупок для каждого покупателя с указанием типа карты:")
purchase_count_per_buyer

Количество покупок для каждого покупателя с указанием типа карты:


,buyer_name,purchase_count,type_pay_card
0,unknown,2427724,Other
1,Jennifer_Williams,2802,Other
2,Michael_Thompson,2306,Other
3,Michael_Smith,2270,UnionPay
4,Matthew_Smith,2249,Other
...,...,...,...
6767,Nathaniel_Bass,481,Visa
6768,Preston_Black,480,Other
6769,Hayley_Andrews,480,AmericanExpress
6770,Natalie_Hurst,474,Other


#### 9. Найти среднее количество покупок в день

In [11]:
average_purchase_per_day = np.round(df.groupby(df['purchase_date'].dt.date)
                                    .size()
                                    .mean(), 0)
print("Среднее количество покупок в день:", average_purchase_per_day)

Среднее количество покупок в день: 4371.0


#### 10. Найти среднее количество покупок в день по типу карты

In [12]:
# Копирование датафрейма
day_df = df.copy()

# Создание столбца с датой без времени
day_df['purchase_date'] = day_df['purchase_date'].dt.date

# Подсчет количества покупок для каждого дня и типа карты
purchase_count_per_day_and_card_type = (
    day_df.groupby(['purchase_date', 'type_pay_card'])
    .size()
    .reset_index(name='count')
)

# Подсчет среднего количества покупок в день для каждого типа карты
mean_purchase_count_per_day_by_card_type = np.round(
    purchase_count_per_day_and_card_type
    .groupby('type_pay_card')['count']
    .mean()
    .reset_index(name='mean_count')
    .sort_values(by='mean_count', ascending=False), 0
)

print("Среднее количество покупок в день по типу карты:")
mean_purchase_count_per_day_by_card_type

Среднее количество покупок в день по типу карты:


,type_pay_card,mean_count
2,Other,1749.0
4,Visa,442.0
0,AmericanExpress,235.0
3,UnionPay,223.0
1,MasterCard,61.0
